IndexFlat 索引是一种基于线性搜索的索引，它通过逐个计算与每个向量的相似度来进行搜索。在数据量较大的时候，搜索效率会较低。此时，我们可以使用 IndexIVFFlat 索引来提升搜索效率。

它的原理如下：对于所有的向量进行聚类，相当于把所有的数据进行分类。当进行查询时，在最相似的 N 个簇中进行线性搜索。这就减少了需要进行相似度计算的数据量，从而提升搜索效率。

需要注意：这种方法是一种在查询的精度和效率之间平衡的方法。N越大->精度大，效率慢；反之亦然

Useful link: https://blog.csdn.net/ResumeProject/article/details/135350945

<img src="02_cluster.png">

In [1]:
import faiss
import numpy as np
import time

# Initialize some random data
np.random.seed(0)
data = np.random.rand(5000000, 256)
ids = np.arange(0, 5000000)
query_vector = np.random.rand(1, 256)

In [23]:
# 1, Linear Index
def test01():
    index = faiss.IndexFlatL2(256)
    index = faiss.IndexIDMap(index)

    # add vector
    index.add_with_ids(data,ids)

    # search vector
    s = time.time()
    D, I = index.search(query_vector, 2)
    print('time of linear', time.time() - s)
    print(D, I)


In [24]:
# 2, Inverted File Index
def test02():
    """
    faiss.IndexIVFFlat() three parameters:
        # 第一个参数：量化器
        # 第二个参数：向量维度
        # 第三个参数：质心数量 (clusters)
    先训练分cluster，然后再linear搜索
    """
    quantizer = faiss.IndexFlatL2(256)
    index = faiss.IndexIVFFlat(quantizer, 256, 100)

    # 指定在搜索时，在最相似的前多少个簇中进行线性搜索。nprobe多，精度高+慢；nprobe少，精度低+快
    index.nprobe = 2

    # Train the input data to get the 100 clusters
    index.train(data)

    # When 'add', means adding the input data into the clusters we trained above.
    index.add(data)

    # Search same query and compare
    s = time.time()
    D, I = index.search(query_vector, 2)
    # 相对于精度最高的linear搜索，IVF为近似搜索，精度不如linear高。
    print('time of IVF', time.time() - s)
    print(D, I)

if __name__ == '__main__':
    test01()
    test02()

time of linear 0.5690667629241943
[[30.425781 30.580658]] [[879992 441435]]
time of IVF 0.014178752899169922
[[31.449919 31.590675]] [[2838142  459745]]
